[Use this website to craft a zero-shot Tiny Time Mixer](https://developer.ibm.com/tutorials/awb-foundation-model-time-series-forecasting/)

Worst comes to worst, you just do a train, val, test split on just your train and validation dataset, splitting your validation dataset evenly between the validation and test splits for the get_datasets function. First try to not include a test split in the dataset splits section, and if that is not an option, then try the prior idea. 

In [2]:
import math
import os
import tempfile

import numpy as np
import pandas as pd
from torch.optim import AdamW
from torch.optim.lr_scheduler import OneCycleLR
from transformers import EarlyStoppingCallback, Trainer, TrainingArguments, set_seed
from transformers.integrations import INTEGRATION_TO_CALLBACK

from tsfm_public import TimeSeriesPreprocessor, TrackingCallback, count_parameters, get_datasets
from tsfm_public.toolkit.get_model import get_model
from tsfm_public.toolkit.lr_finder import optimal_lr_finder
from tsfm_public.toolkit.visualization import plot_predictions
from tsfm_public.models.tinytimemixer import TinyTimeMixerForPrediction

In [21]:
# Dataset
timestamp_column = "date"
id_columns = []  # mention the ids that uniquely identify a time-series.

# Understanding the split config -- slides
split_config = {
    "train": [0, 24],
    "valid": [24, 32],
    "test": [
        32,
        40,
    ],
}


def generate_time_series_data(start_time: str = "2021-01-01 00:00:00", 
                              freq: str = "H", 
                              periods: int = 40, 
                              n_variables: int = 10, 
                              univariate: bool = False) -> pd.DataFrame:
    """
    Generate a univariate or multivariate time series DataFrame.
    
    Args:
        start_time (str): Start datetime.
        freq (str): Pandas frequency string (e.g., 'H' for hourly).
        periods (int): Number of time steps.
        n_variables (int): Number of variables (ignored if univariate=True).
        univariate (bool): Whether to generate univariate (single-column) data.

    Returns:
        pd.DataFrame: Time series data with datetime index.
    """
    index = pd.date_range(start=start_time, periods=periods, freq=freq)

    if univariate:
        data = np.random.randn(periods)
        df = pd.DataFrame(data, index=index, columns=["A"])
    else:
        columns = list("ABCDEFGHIJ")[:n_variables]
        data = np.random.randn(periods, n_variables)
        df = pd.DataFrame(data, index=index, columns=columns)
    
    return df

df_multivariate = generate_time_series_data()
df_multivariate.index.rename("date", inplace=True)
df_multivariate.reset_index(inplace=True)
print(df_multivariate.head())
df_univariate = generate_time_series_data(periods=800, univariate=True)
df_univariate.index.rename("date", inplace=True)
df_univariate.reset_index(inplace=True)
print(df_univariate)

                 date         A         B         C         D         E  \
0 2021-01-01 00:00:00  0.166708 -1.327301 -0.296454  0.332246 -0.458729   
1 2021-01-01 01:00:00  0.647631  0.235870  0.491203 -0.311199  0.592118   
2 2021-01-01 02:00:00  0.499633 -0.381601 -0.043697  0.982387  0.268937   
3 2021-01-01 03:00:00 -1.296820  0.720114  0.609906  0.918570 -1.222829   
4 2021-01-01 04:00:00 -1.458164 -0.584174 -1.053504 -0.690108 -0.123941   

          F         G         H         I         J  
0  0.277829  0.688036 -0.613306  0.060988 -1.251947  
1 -1.242010  0.895689 -0.153702  0.976559 -1.395857  
2  1.925392  0.817282  0.332467  0.560036 -0.061332  
3  0.116264 -0.969458  0.664861  0.467591  0.587565  
4  1.811229 -0.114547  0.615304  1.778976  1.483378  
                   date         A
0   2021-01-01 00:00:00 -0.576390
1   2021-01-01 01:00:00  0.715841
2   2021-01-01 02:00:00 -0.052029
3   2021-01-01 03:00:00  1.445212
4   2021-01-01 04:00:00 -1.132915
..                  .

/var/folders/ch/c2hzh4rs0vq5mjq5cp46s_hh0000gn/T/ipykernel_14332/720629148.py:34: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  index = pd.date_range(start=start_time, periods=periods, freq=freq)
/var/folders/ch/c2hzh4rs0vq5mjq5cp46s_hh0000gn/T/ipykernel_14332/720629148.py:34: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  index = pd.date_range(start=start_time, periods=periods, freq=freq)


In [24]:
zeroshot_model = TinyTimeMixerForPrediction.from_pretrained("ibm/TTM", revision="main", prediction_filter_length=24)
from tsfm_public.toolkit.time_series_forecasting_pipeline import TimeSeriesForecastingPipeline

timestamp_column = "date"
target_columns = ["A"]

column_specifiers = {
    "timestamp_column": timestamp_column,
    "id_columns": id_columns,
    "target_columns": ["A"],
    "control_columns": [],
}

context_length = 512
forecast_length = 24

tsp = TimeSeriesPreprocessor(
        **column_specifiers,
        context_length=context_length,
        prediction_length=forecast_length,
        scaling=True,
        encode_categorical=False,
        scaler_type="standard",
    )

df_uni_standardized = tsp._standardize_dataframe(df_univariate)
tsp.train(df_uni_standardized)

zs_forecast_pipeline = TimeSeriesForecastingPipeline(
    model=zeroshot_model,
    device="cpu",
    timestamp_column=timestamp_column,
    id_columns=[],
    target_columns=target_columns,
    freq="H",
    context_length=context_length,
    prediction_length=forecast_length
)

x = tsp.preprocess(df_univariate)
print("Preprocessed shape:", x.shape)
print("Preprocessed index:", x.index)
print(x.head())
x.index = df_univariate["date"][:len(x)]
assert not x[target_columns].isnull().any().any(), "NaNs in target column after preprocessing"

forecast = zs_forecast_pipeline(x)
print(forecast.values)

Device set to use cpu


Preprocessed shape: (800, 2)
Preprocessed index: Index([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
       ...
       790, 791, 792, 793, 794, 795, 796, 797, 798, 799],
      dtype='int64', length=800)
                 date         A
0 2021-01-01 00:00:00 -0.539418
1 2021-01-01 01:00:00  0.770339
2 2021-01-01 02:00:00 -0.007945
3 2021-01-01 03:00:00  1.509602
4 2021-01-01 04:00:00 -1.103490
[[Timestamp('2021-01-22 07:00:00')
  list([0.05359385535120964, -0.026165656745433807, 0.028742067515850067, -0.01773480698466301, -0.020476259291172028, -0.015296483412384987, -0.06655223667621613, -0.04014850780367851, -0.041719209402799606, 0.05354096367955208, -0.0027126320637762547, -0.009542293846607208, 0.006410212256014347, 0.03776101395487785, -0.0790354534983635, 0.07656426727771759, -0.030412763357162476, 0.10390327870845795, -0.038141604512929916, -0.050918202847242355, -0.008571170270442963, 0.003067891113460064, 0.12052319198846817, -0.03052983619272709])
  list([-1.0650275945663

In [4]:
def zeroshot_eval(data, batch_size, context_length=512, forecast_length=96, column_specifiers={}):
    # Get data

    tsp = TimeSeriesPreprocessor(
        **column_specifiers,
        context_length=context_length,
        prediction_length=forecast_length,
        scaling=True,
        encode_categorical=False,
        scaler_type="standard",
    )

    # Load model
    zeroshot_model = get_model(
        TTM_MODEL_PATH,
        context_length=context_length,
        prediction_length=forecast_length,
        freq_prefix_tuning=False,
        freq=None,
        prefer_l1_loss=False,
        prefer_longer_context=True,
    )

    dset_train, dset_valid, dset_test = get_datasets(
        tsp, data, split_config, use_frequency_token=zeroshot_model.config.resolution_prefix_tuning
    )

    temp_dir = tempfile.mkdtemp()
    # zeroshot_trainer
    zeroshot_trainer = Trainer(
        model=zeroshot_model,
        args=TrainingArguments(
            output_dir=temp_dir,
            per_device_eval_batch_size=batch_size,
            seed=SEED,
            report_to="none",
        ),
    )
    # evaluate = zero-shot performance
    print("+" * 20, "Test MSE zero-shot", "+" * 20)
    zeroshot_output = zeroshot_trainer.evaluate(dset_test)
    print(zeroshot_output)

    # get predictions

    predictions_dict = zeroshot_trainer.predict(dset_test)

    predictions_np = predictions_dict.predictions[0]

    print(np.transpose(predictions_np[0]))
    print(np.transpose(predictions_np[0]).shape)

    # get backbone embeddings (if needed for further analysis)

    backbone_embedding = predictions_dict.predictions[1]

    print(backbone_embedding[0])
    print(backbone_embedding[0].shape)

    # plot
    """
    plot_predictions(
        model=zeroshot_trainer.model,
        dset=dset_test,
        plot_dir=os.path.join(OUT_DIR, dataset_name),
        plot_prefix="test_zeroshot",
        indices=[685, 118, 902, 1984, 894, 967, 304, 57, 265, 1015],
        channel=0,
    )"""

In [102]:
# Multivariate Tiny Time Mixer
column_specifiers = {
    "timestamp_column": timestamp_column,
    "id_columns": id_columns,
    "target_columns": ["A", "B", "C", "D", "E", "F", "G", "H", "I", "J"],
    "control_columns": [],
}
zeroshot_eval(
    data=df_multivariate, context_length=CONTEXT_LENGTH, forecast_length=PREDICTION_LENGTH, batch_size=8, column_specifiers=column_specifiers
)

INFO:p-3927:t-8034164416:get_model.py:get_model:Loading model from: ibm-granite/granite-timeseries-ttm-r2
INFO:p-3927:t-8034164416:get_model.py:get_model:Model loaded successfully from ibm-granite/granite-timeseries-ttm-r2, revision = 52-16-ft-r2.1.
INFO:p-3927:t-8034164416:get_model.py:get_model:[TTM] context_length = 52, prediction_length = 16
/opt/miniconda3/envs/ttm/lib/python3.10/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


++++++++++++++++++++ Test MSE zero-shot ++++++++++++++++++++


{'eval_loss': 1.2032523155212402, 'eval_model_preparation_time': 0.0039, 'eval_runtime': 0.2653, 'eval_samples_per_second': 3.769, 'eval_steps_per_second': 3.769}
[[ 0.04402132  0.03331154  0.02133745  0.00305077 -0.00776591 -0.00895865
   0.00161481]
 [ 0.10501055  0.01142201 -0.01504199 -0.03931901 -0.03481245 -0.02803831
  -0.01688247]
 [-0.33105555 -0.22063375 -0.14527488 -0.14149031 -0.09604315 -0.05538443
  -0.04476943]
 [ 0.8807009   0.50128347  0.33325475  0.25177714  0.19909978  0.17655475
   0.16773659]
 [-0.26294658 -0.18029238 -0.12163518 -0.08381483 -0.07242465 -0.05933443
  -0.05498382]
 [-0.95935035 -0.75746226 -0.67643684 -0.5775811  -0.5128267  -0.50890464
  -0.50662965]
 [ 0.65836704  0.4228226   0.3083427   0.25424948  0.23301637  0.22384226
   0.22057135]
 [ 0.12895179  0.00613056 -0.016215   -0.03495287 -0.03996248 -0.03250063
  -0.01976608]
 [ 0.6290895   0.4039339   0.29543215  0.23577464  0.21004614  0.21258509
   0.19800416]
 [ 0.23581614  0.16492547  0.1374643

In [103]:
# Univariate Tiny Time Mixer
column_specifiers = {
    "timestamp_column": timestamp_column,
    "id_columns": id_columns,
    "target_columns": ["A"],
    "control_columns": [],
}
zeroshot_eval(
    data=df_univariate, context_length=CONTEXT_LENGTH, forecast_length=PREDICTION_LENGTH, batch_size=8, column_specifiers=column_specifiers
)

INFO:p-3927:t-8034164416:get_model.py:get_model:Loading model from: ibm-granite/granite-timeseries-ttm-r2
INFO:p-3927:t-8034164416:get_model.py:get_model:Model loaded successfully from ibm-granite/granite-timeseries-ttm-r2, revision = 52-16-ft-r2.1.
INFO:p-3927:t-8034164416:get_model.py:get_model:[TTM] context_length = 52, prediction_length = 16
/opt/miniconda3/envs/ttm/lib/python3.10/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


++++++++++++++++++++ Test MSE zero-shot ++++++++++++++++++++


{'eval_loss': 1.6534693241119385, 'eval_model_preparation_time': 0.0041, 'eval_runtime': 0.1014, 'eval_samples_per_second': 9.858, 'eval_steps_per_second': 9.858}
[[-0.31559467 -0.26337448 -0.2292342  -0.22169566 -0.17298758 -0.13871092
  -0.11940026]]
(1, 7)
[[[ 2.58540541e-01  1.89691231e-01 -9.90554392e-01 -1.80186749e+00
    1.37157008e-01  2.34319940e-01 -1.92468357e+00  1.77439868e-01
    3.82961571e-01  9.24913809e-02 -7.71513462e-01 -6.02124631e-01
   -9.41056848e-01 -8.00155044e-01 -8.31349969e-01 -1.80217361e+00
   -1.32850003e+00 -1.07328176e+00 -1.23185384e+00  6.71325848e-02]
  [ 1.04538046e-01 -5.37322089e-03 -2.67058402e-01 -4.80283797e-01
    4.71760929e-02  4.77122851e-02 -4.43057716e-01  8.26564506e-02
   -4.81275469e-02  3.16931754e-02  1.04031339e-02 -2.94233203e-01
   -1.69283167e-01 -5.34338057e-01  4.30340230e-01 -4.40655589e-01
   -1.79318916e-02 -3.66196424e-01 -1.23684146e-01 -2.55382862e-02]
  [ 9.88186970e-02  1.78145003e-02 -1.88574865e-01 -4.13320422e-01
 